## Spread Analysis of Co-integrated pairs for Mean Reversion Trading

- Author: Ayush Patel
- Github: https://github.com/Ayush-Patel15 
- LinkedIn: https://www.linkedin.com/in/ayush-15-patel/

<hr>

In financial markets, prices often deviate from their typical patterns due to market noise, sentiment, or external factors. But what if we could identify relationships between assets that remain stable over time despite these short-term fluctuations? This post continues the journey into identifying such relationships using statistical methods and explores how these insights can be turned into actionable trading strategies.

### Analysis of Cointegrated Pairs

To learn about cointegration, and related terms, you can read my previous post on cointegration.

The goal of this analysis is to build upon the results of the cointegration test conducted earlier by exploring and modeling the relationship between cointegrated stock pairs. We aim to examine their spread (any linear relationship between them), assess stationarity, and use statistical tools to generate actionable insights for pairs trading strategies.

<hr>

### Pairs Trading and the Mean Reversion Strategy

Pairs trading is a market-neutral strategy that profits from temporary deviations between two related assets. When two stocks exhibit cointegration, their prices move together in the long term, even if they temporarily deviate from their typical relationship. The key idea is to identify these deviations, assuming the spread between them will revert to its historical mean. Here’s how it works:

- Identifying a Pair: Two cointegrated stocks, let’s call them Stock A and Stock B, have a stable spread between their prices over time.
- Monitoring the Spread: If the spread widens significantly (e.g., Z-score > +2SD), Stock A may be considered overvalued, and Stock B undervalued. Spread is just any linear relationship between the Stock A and Stock B.
- Trading Signals: Go Long (Buy) on the undervalued stock and Go Short (Sell) on the overvalued stock when the spread deviates significantly. Close the trade when the spread reverts to the mean, realizing a profit.
- Risk Management: By pairing long and short positions, pairs trading minimizes exposure to broader market trends and focuses on the relationship between the two assets.

<hr>

### Importance in Finance

- Market Neutral Strategy: Pairs trading strategies are market-neutral, meaning they are unaffected by overall market movements.
- Identifying Arbitrage Opportunities: When the spread deviates significantly from its mean, it can signal opportunities for profitable trades.
- Risk Management: Stationary spreads allow for clearly defined entry and exit points, helping manage trade risks.

<hr>

### Python Implementation

Let's dive directly to the main topic, and learn the steps involved in research of finding Mean Reverting pairs. For this demonstration, I am using the output of one of my previous post, where I have saved a dataframe with the pairs and their verdict of `cointegrated` or `not-cointegrated`. I will filter out the pairs that are cointegrated and follow the below steps.

`STEP-1`: Use OLS (Ordinary Least Squares) Regression to find the relationship between the pairs.

- Apply an OLS regression to fit a linear model between the two stocks in each cointegrated pair.
- The regression equation models one stock as a function of the other, helping compute the spread between them.

`STEP-2`: Create the Spread using the Beta/Slope coeficient of the model.

- In pairs trading, this beta is called as the hedge ratio.
- Compute the spread as the linear equation: stock_y - (hedge_ratio * stock_x), where x and y are the dependent and independent variables.
- The spread represents the linear combination of their expected relationship.

`STEP-3`: Apply the stationarity test on the created spread series.

- Use the Augmented Dickey-Fuller (ADF) test to verify if the spread is stationary.
- A stationary spread is crucial for pairs trading, as it suggests the spread will revert to its historical mean over time.

`STEP-4`: If the spread is stationary, generate the Z-Score Series

- Normalize the spread by calculating its Z-score to measure how far it deviates from the mean.
- This step helps define boundaries for typical versus extreme deviations.
$$ Z(t) = (Spread(t) - \mu_s) / \sigma_s $$

Where $\mu_s$ is the mean and $\sigma_s$ is the standard deviation of the spread.

`STEP-5`: Plot the Z-Score Series, and save the output.

- Visualize the Z-score series with statistical boundaries at the mean, +2SD, and -2SD.
- Highlight key points where the spread crosses these thresholds to identify potential trading signals.

This plot have the pairs to analyze and decide, whether to trade or apply the mean reversion pairs trading logic or not.

In [9]:
## Necessary Import Statements
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import os

In [10]:
## FOLDER-PATHS
DATA_FOLDERPATH = "E:\\Market-Work\\All-Data\\Daily_Data-Stocks_Indices\\"
VERDICT_FILEPATH = "E:\\Market-Work\\My-Work\\Cointegration_Test\\Cointegration-Verdict.csv"
ZSCORE_FOLDERPATH = "E:\\Market-Work\\My-Work\\Spread_Analysis-and-Mean_Reversion_Technique\\Mean_Reverting_Pairs"

## Reading of the dataframe, saved previously
verdict_df = pd.read_csv(VERDICT_FILEPATH, index_col=0)

In [11]:
## Function to get the pair names, that are cointegrated, do remove the duplicates: apply sorting and then append to the set
def get_cointegrated_pairs(verdict_df):
	cointegrated_sets = set()
	for stock_1 in verdict_df.columns:
		cointegrated = verdict_df[(verdict_df[stock_1]=="cointegrated")][stock_1]
		for stock_2 in cointegrated.index:				
			pair_list = [stock_1, stock_2]
			pair_list.sort()
			cointegrated_sets.add(tuple(pair_list))
	cointegrated_pair = list(cointegrated_sets)
	return cointegrated_pair

In [12]:
## Function to take a cointegrated pair, create a spread and then check for stationarity in the spread
def save_zscore_plots(stock_pair, DATA_FOLDERPATH, ZSCORE_FOLDERPATH):
	stock_1, stock_2 = stock_pair
	stockname_1, stockname_2 = stock_1.split(".csv")[0], stock_2.split(".csv")[0]
	# print("==> Processing:", stock_1, stock_2)
	stock_filepath_1 = os.path.join(DATA_FOLDERPATH, stock_1)
	stock_filepath_2 = os.path.join(DATA_FOLDERPATH, stock_2)
	stock_df_1 = pd.read_csv(stock_filepath_1, index_col=0, usecols=["Date","Close"])
	stock_df_1 = stock_df_1.loc["2021":]
	stock_df_2 = pd.read_csv(stock_filepath_2, index_col=0, usecols=["Date","Close"])
	stock_df_2 = stock_df_2.loc["2021":]
	## Fit the data with OLS method
	y_series = np.array(stock_df_1["Close"])
	x_series = np.array(stock_df_2["Close"])
	model = sm.OLS(y_series, x_series)
	result = model.fit()
	hedge_ratio = result.params[0]
	## Build the equation for spread, and check for stationarity of that spread
	spread = y_series - (hedge_ratio * x_series)
	spread_pvalue = adfuller(spread)[1]
	## If the spread is stationary, plot and save the Z-Score of the spread series
	if (spread_pvalue < 0.05):
		print(f"{stockname_1}-{stockname_2} is stationary series.")
		z_spread = (spread - np.mean(spread)) / np.std(spread)
		zscore_mean, zscore_std = np.mean(z_spread), np.std(z_spread)
		plt.figure(figsize=(12, 10))
		plt.plot(z_spread, color="black", label="Z-score")
		plt.title(f"{stockname_1}-{stockname_2}, Hedge:{hedge_ratio}, p-value:{spread_pvalue}")
		plt.axhline(y=zscore_mean, color="blue", linestyle="dashed", label="Mean", linewidth=2)
		plt.axhline(y=zscore_mean + (2 * zscore_std), color="red", linestyle="dotted", label="+2 SD", linewidth=2)
		plt.axhline(y=zscore_mean - (2 * zscore_std), color="green", linestyle="dotted", label="-2 SD", linewidth=2)
		plt.legend()
		plt.tight_layout()
		saving_path = os.path.join(ZSCORE_FOLDERPATH, f"{stockname_1}-{stockname_2}.png")
		plt.savefig(saving_path)
		plt.close()

In [13]:
## Take out all the pairs from the dataframe
cointegrated_pair = get_cointegrated_pairs(verdict_df)
cointegrated_pair.sort()

In [14]:
## Save the plots, using the zscore function
for stock_pair in cointegrated_pair:
	save_zscore_plots(stock_pair, DATA_FOLDERPATH, ZSCORE_FOLDERPATH)

ADANIPORTS-CIPLA is stationary series.
ADANIPORTS-HEROMOTOCO is stationary series.
ADANIPORTS-ONGC is stationary series.
APOLLOHOSP-CIPLA is stationary series.
APOLLOHOSP-DRREDDY is stationary series.
APOLLOHOSP-GRASIM is stationary series.
APOLLOHOSP-HCLTECH is stationary series.
APOLLOHOSP-ICICIBANK is stationary series.
APOLLOHOSP-NIFTY50 is stationary series.
APOLLOHOSP-ULTRACEMCO is stationary series.
AXISBANK-BRITANNIA is stationary series.
AXISBANK-CIPLA is stationary series.
AXISBANK-DRREDDY is stationary series.
AXISBANK-MARUTI is stationary series.
BAJAJFINSV-BAJFINANCE is stationary series.
BANKNIFTY-BRITANNIA is stationary series.
BANKNIFTY-DRREDDY is stationary series.
BHARTIARTL-SUNPHARMA is stationary series.
CIPLA-EICHERMOT is stationary series.
CIPLA-GRASIM is stationary series.
CIPLA-ICICIBANK is stationary series.
CIPLA-SBIN is stationary series.
COALINDIA-NTPC is stationary series.
DRREDDY-ULTRACEMCO is stationary series.
EICHERMOT-ICICIBANK is stationary series.
EI

<hr>

### Some Z-Score plots for reference

All the plots are present at my github, in the folder Mean_Reverting_Pairs.

<img src="Mean_Reverting_Pairs/AXISBANK-BRITANNIA.png" height=1000, width=1000>
<img src="Mean_Reverting_Pairs/BAJAJFINSV-BAJFINANCE.png" height=500, width=500>
<img src="Mean_Reverting_Pairs/ONGC-POWERGRID.png" height=500, width=500>

<hr>

### Limitations of Mean Reversion Pairs Trading

- Assumption of Stationarity: The success of pairs trading hinges on the assumption that the spread between cointegrated pairs is stationary and will revert to its mean over time. However, market conditions can change, causing previously stationary relationships to break down.

- Selection Bias: Choosing the right pairs is critical. Poor pair selection can lead to false signals and suboptimal trading outcomes. The process often involves extensive data analysis and rigorous testing to ensure robust results.

- Market Regimes and Structural Shifts: Unexpected macroeconomic events or structural changes in industries can permanently shift relationships between pairs, rendering historical patterns invalid.

- Execution Risk: Delays in trade execution can result in missed opportunities or increased losses. High-frequency market movements may reduce the efficiency of pairs trading strategies.

- Correlation vs. Cointegration: Pairs trading requires cointegration, not just correlation. Two highly correlated stocks may not maintain a stable spread, leading to potential strategy failure if cointegration is not carefully verified.

- Transaction Costs and Slippage: Frequent trading to capture small deviations can result in significant transaction costs and slippage, eroding potential profits.

<hr>

### Recommendation for Better Results

To achieve more reliable and optimal outcomes, consider applying this technique to ETFs, related commodities, or sector-wise stock constituents, as these groups often share common factors that influence their movements, increasing the likelihood of stable relationships. Please note that this document focuses primarily on explaining the technique and steps involved, therefore I have used the data of all Nifty50 stocks that I had from the past cointegration test.

<hr>

`Disclaimer`: This is just for the purpose of knowledge and my own learning, and not any advice to create any kind of trading strategy based on the post. Do you own analysis and invest in markets. However, this is just for my own learning purpose, so need not to worry. For learning purpose, I can test for any weird possibilities, even if doesn't make any sense in the practical world.

Do let me know, if you want more details on the spread and mean reversion pair trading strategy.

<hr>